In [7]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np

In [3]:
df = pd.read_csv('IMDB.csv')
df = df.sample(500)
df.to_csv('data.csv', index=False)
df.head()

,review,sentiment
168,"Mmmm, a previous summary says ""if you like ali...",negative
139,Bottom-of-the-barrel stinker is so bad it's be...,negative
855,What the hell was all that about? I saw The Ma...,negative
660,Where was this film when I was a kid? After hi...,positive
749,"Ya know, I have no idea how everybody else's t...",negative


In [4]:
# data preprocessing

# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(remove_stop_words)
        df['review'] = df['review'].apply(removing_numbers)
        df['review'] = df['review'].apply(removing_punctuations)
        df['review'] = df['review'].apply(removing_urls)
        df['review'] = df['review'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

In [11]:
df = normalize_text(df)
df.head()

,review,sentiment
168,mmmm previous summary say like alien predator ...,negative
139,bottom barrel stinker bad beyond funny plot am...,negative
855,hell saw matrix amazed spectacular movie ever ...,negative
660,film kid parent split tadashi move mom live gr...,positive
749,ya know idea everybody else teenage life refle...,negative


In [12]:
df['sentiment'].value_counts()

sentiment
negative    251
positive    249
Name: count, dtype: int64

In [13]:
x = df['sentiment'].isin(['positive','negative'])
df = df[x]

In [14]:
df['sentiment'] = df['sentiment'].map({'positive':1, 'negative':0})
df.head()

,review,sentiment
168,mmmm previous summary say like alien predator ...,0
139,bottom barrel stinker bad beyond funny plot am...,0
855,hell saw matrix amazed spectacular movie ever ...,0
660,film kid parent split tadashi move mom live gr...,1
749,ya know idea everybody else teenage life refle...,0


In [15]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [25]:
vectorizer = CountVectorizer(max_features=50)
X = vectorizer.fit_transform(df['review'])
y = df['sentiment']

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [27]:
import dagshub

mlflow.set_tracking_uri('https://dagshub.com/pranavdhebe93/MLOps-Proj-2.mlflow')
dagshub.init(repo_owner='pranavdhebe93', repo_name='MLOps-Proj-2', mlflow=True)

# mlflow.set_experiment("Logistic Regression Baseline")
mlflow.set_experiment("Logistic Regression Baseline")


2025-12-19 18:44:41,255 - INFO - HTTP Request: GET https://dagshub.com/api/v1/repos/pranavdhebe93/MLOps-Proj-2 "HTTP/1.1 200 OK"


Initialized MLflow to track repo "pranavdhebe93/MLOps-Proj-2"

2025-12-19 18:44:41,259 - INFO - Initialized MLflow to track repo "pranavdhebe93/MLOps-Proj-2"


Repository pranavdhebe93/MLOps-Proj-2 initialized!

2025-12-19 18:44:41,261 - INFO - Repository pranavdhebe93/MLOps-Proj-2 initialized!


<Experiment: artifact_location='mlflow-artifacts:/a67b76ee388d4beaa4bfdfb915543a49', creation_time=1766149683776, experiment_id='0', last_update_time=1766149683776, lifecycle_stage='active', name='Logistic Regression Baseline', tags={'mlflow.experimentKind': 'custom_model_development'}>

In [28]:
import mlflow
import logging
import os
import time
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

logging.info("Starting MLflow run...")

with mlflow.start_run():
    start_time = time.time()
    
    try:
        logging.info("Logging preprocessing parameters...")
        mlflow.log_param("vectorizer", "Bag of Words")
        mlflow.log_param("num_features", 50)
        mlflow.log_param("test_size", 0.25)

        logging.info("Initializing Logistic Regression model...")
        model = LogisticRegression(max_iter=1000)  # Increase max_iter to prevent non-convergence issues

        logging.info("Fitting the model...")
        model.fit(X_train, y_train)
        logging.info("Model training complete.")

        logging.info("Logging model parameters...")
        mlflow.log_param("model", "Logistic Regression")

        logging.info("Making predictions...")
        y_pred = model.predict(X_test)

        logging.info("Calculating evaluation metrics...")
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        logging.info("Logging evaluation metrics...")
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

        logging.info("Saving and logging the model...")
        mlflow.sklearn.log_model(model, "model")

        # Log execution time
        end_time = time.time()
        logging.info(f"Model training and logging completed in {end_time - start_time:.2f} seconds.")

        # Save and log the notebook
        # notebook_path = "exp1_baseline_model.ipynb"
        # logging.info("Executing Jupyter Notebook. This may take a while...")
        # os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
        # mlflow.log_artifact(notebook_path)

        # logging.info("Notebook execution and logging complete.")

        # Print the results for verification
        logging.info(f"Accuracy: {accuracy}")
        logging.info(f"Precision: {precision}")
        logging.info(f"Recall: {recall}")
        logging.info(f"F1 Score: {f1}")

    except Exception as e:
        logging.error(f"An error occurred: {e}", exc_info=True)


2025-12-19 18:44:42,725 - INFO - Starting MLflow run...
2025-12-19 18:44:43,120 - INFO - Logging preprocessing parameters...
2025-12-19 18:44:44,240 - INFO - Initializing Logistic Regression model...
2025-12-19 18:44:44,240 - INFO - Fitting the model...
2025-12-19 18:44:44,253 - INFO - Model training complete.
2025-12-19 18:44:44,254 - INFO - Logging model parameters...
2025-12-19 18:44:44,644 - INFO - Making predictions...
2025-12-19 18:44:44,646 - INFO - Calculating evaluation metrics...
2025-12-19 18:44:44,649 - INFO - Logging evaluation metrics...
2025-12-19 18:44:49,853 - INFO - Saving and logging the model...
2025/12/19 18:44:49 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/12/19 18:45:01 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025-12-19 18:45:05,787 - INFO - Model training and logging completed in 2

🏃 View run fortunate-sponge-675 at: https://dagshub.com/pranavdhebe93/MLOps-Proj-2.mlflow/#/experiments/0/runs/87dd58c7584841aa87cb14299af3ff56
🧪 View experiment at: https://dagshub.com/pranavdhebe93/MLOps-Proj-2.mlflow/#/experiments/0
